In [ ]:
!pip install ultralytics roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.1/875.1 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


In [ ]:
from ultralytics import YOLO

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="48UEsVrR5g6o1ZqmM9tq")
project = rf.workspace("projects-6igvl").project("face-mask-detection-960jp")
version = project.version(1)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.103, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Face-mask-detection-1 in yolov8:: 100%|██████████| 1708/1708 [00:00<00:00, 3821.72it/s]


In [ ]:
dataset.location

'/content/Face-mask-detection-1'

In [ ]:
# Train the model on the dataset
model = YOLO('yolov8n.pt')
model.train(data='/content/Face-mask-detection-1/data.yaml', epochs=50, imgsz=640)


Ultralytics YOLOv8.2.103 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/Face-mask-detection-1/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_

KeyboardInterrupt: 

In [ ]:
import cv2
from ultralytics import YOLO

# Load the trained YOLOv8 model
model = YOLO('best (1).pt')  # Replace 'best.pt' with your model file path if necessary

# Define the class names and colors for each class
class_names = ['incorrect_weard_mask', 'with_mask', 'without_mask']  # Correct order

colors = {
    'with_mask': (0, 255, 0),           # Green for 'with_mask'
    'incorrect_weard_mask': (0, 255, 255),  # Yellow for 'incorrect_weard_mask'
    'without_mask': (0, 0, 255)         # Red for 'without_mask'
}

# Start capturing video from the webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()  # Read frame from the camera
    if not ret:
        break

    # Run YOLOv8 inference on the frame
    results = model(frame)[0]  # Get the first result from the list

    # Loop through each detection and draw bounding boxes for all
    for box in results.boxes:  # Loop through the boxes detected
        # Extracting box coordinates and confidence
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()  # Get the bounding box coordinates
        conf = box.conf.cpu().item()  # Get confidence
        cls = int(box.cls.cpu().item())  # Get class index

        # Get class name and corresponding color
        class_name = class_names[cls]
        color = colors[class_name]

        # Draw the bounding box around the face
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)

        # Add label for the prediction
        label = f'{class_name}: {conf:.2f}'
        cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    # Display the result in a window
    cv2.imshow("Face Mask Detection", frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the window
cap.release()
cv2.destroyAllWindows()